In [ ]:
!pip install numpy pandas scikit-learn catboost langchain-openai langchain

In [5]:
import numpy as np
import pandas as pd
import os

from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [24]:
train_df = shuffle(pd.read_csv('Training.csv').iloc[:, :-1])
test_df = pd.read_csv('Testing.csv')

train_df.reset_index().head(5)

index  itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  \
0   2414        0          0                     0                    0   
1   3378        1          1                     0                    0   
2   2235        0          0                     0                    0   
3    201        1          0                     0                    0   
4   4860        0          0                     0                    0   

   shivering  chills  joint_pain  stomach_pain  acidity  ...  blackheads  \
0          0       0           0             0        0  ...           0   
1          0       0           0             0        0  ...           0   
2          0       1           0             0        0  ...           0   
3          0       0           0             0        0  ...           0   
4          0       0           1             0        0  ...           0   

   scurring  skin_peeling  silver_like_dusting  small_dents_in_nails  \
0         0             0                    0                     0   
1         0             0                    0                     0   
2         0             0                    0                     0   
3         0             0                    0                     0   
4         0             0                    0                     0   

   inflammatory_nails  blister  red_sore_around_nose  yellow_crust_ooze  \
0                   0        0                     0                  0   
1                   0        0                     0                  0   
2                   0        0                     0                  0   
3                   0        0                     0                  0   
4                   0        0                     0                  0   

                                 prognosis  
0  (vertigo) Paroymsal  Positional Vertigo  
1                              Chicken pox  
2                                  Typhoid  
3                              Hepatitis B  
4                              Hepatitis D  

[5 rows x 134 columns]

In [27]:
train_x = train_df.iloc[:, :-1]
train_y = train_df.iloc[:, -1]

test_x = test_df.iloc[:, :-1]
test_y = test_df.iloc[:, -1]

['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis', 'Drug Reaction']

# Targets

In [25]:
print(test_y.unique().tolist())

['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction'
 'Peptic ulcer diseae' 'AIDS' 'Diabetes ' 'Gastroenteritis'
 'Bronchial Asthma' 'Hypertension ' 'Migraine' 'Cervical spondylosis'
 'Paralysis (brain hemorrhage)' 'Jaundice' 'Malaria' 'Chicken pox'
 'Dengue' 'Typhoid' 'hepatitis A' 'Hepatitis B' 'Hepatitis C'
 'Hepatitis D' 'Hepatitis E' 'Alcoholic hepatitis' 'Tuberculosis'
 'Common Cold' 'Pneumonia' 'Dimorphic hemmorhoids(piles)' 'Heart attack'
 'Varicose veins' 'Hypothyroidism' 'Hyperthyroidism' 'Hypoglycemia'
 'Osteoarthristis' 'Arthritis' '(vertigo) Paroymsal  Positional Vertigo'
 'Acne' 'Urinary tract infection' 'Psoriasis' 'Impetigo']


In [9]:
model = CatBoostClassifier(iterations=50, learning_rate=0.1, depth=2, loss_function='MultiClass', verbose=False)
model.fit(train_x, train_y)

predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9761904761904762


In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key="")

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert in medical science."),
        ("human", "Please provide general prescriptions for {user_input}"),
    ]
)

chain = prompt | llm

In [32]:
def expert_advice():
    disease=model.predict(train_x.head(1))
    print("Judging from symptoms, you are most likely facing:",disease.tolist())
    print()
    response=chain.invoke({"user_input": disease})
    print(response.content)

In [33]:
expert_advice()

Judging from symptoms, you are most likely facing: [['(vertigo) Paroymsal  Positional Vertigo']]

Paroxysmal Positional Vertigo (PPV) is a common type of vertigo that is typically triggered by changes in head position. The following general prescriptions are commonly recommended for managing PPV:

1. Epley maneuver: This is a series of head movements that can help reposition the displaced crystals in the inner ear that are causing the vertigo. A healthcare provider or physical therapist can demonstrate how to perform the Epley maneuver correctly.

2. Medications: In some cases, medications such as meclizine or dimenhydrinate may be prescribed to help alleviate symptoms of vertigo. These medications can help reduce dizziness and nausea.

3. Vestibular rehabilitation therapy: This is a type of physical therapy that focuses on exercises and maneuvers to improve balance and reduce symptoms of vertigo. A physical therapist can create a customized exercise program to help manage PPV.

4. Lif